此  .ipynb  把测试集代码的 ast 进行归因贡献值计算，并且对父节点(AST_statement)的不同类型。根据统计分析结果。
<br>打印出 <br>
[('MethodDeclaration', 0.022379315476190476)]<br>
[('树级语句类型','贡献值')]<br>
0<br>
索引<br>
挑出实例，不用挑太多

将每个实例中的类别贡献值与统计分析中的绝对值均值进行大小比较，大于绝对值均值的保留

In [263]:
import javalang
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, TensorDataset


from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

# 解决服务器挂掉的问题
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [264]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device( "cpu")

In [265]:
MAX_LEN = 1024
MAX_EPOCHS = 10
BATCH_SIZE = 4
LEARNING_RATE = 1e-5
NUM_CLASSES = 2
WEIGTH_DECAY = 1e-3

In [266]:
def retrieve_test_data():
    test_param = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/param/test.json")
    test_return = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/return/test.json")
    test_summary = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/summary/test.json")
    test_df = pd.concat([test_summary,test_param, test_return], axis=0)
    test_df = test_df[:20]
    test_df = test_df.reset_index(drop=True)
    return test_df
test_df = retrieve_test_data()

In [267]:
len(test_df)

20

In [268]:
# 用这个，这个对了
def get_blocks_improved_BFS_code(root):
    queue = []
    nodes = []
    queue.append(root)
    level = 0
    while queue:
        level = level + 1
        for i in range(len(queue)):
            node = queue.pop(0)
            nodes.append(node)
            if hasattr(node, 'children'):
                for child in node.children:
                    queue.append(child)
            else:
                try:
                    len_node = len(node)
                    for i in range(len_node):
                        if hasattr(node[i], 'children'):
                            for child in node[i].children:
                                 queue.append(child)
                except Exception as e:
                    continue
    for node in nodes:
        if isinstance(node, javalang.tree.Node):
            index = nodes.index(node)
            nodes[index] = type(node).__name__
        if isinstance(node, list):
            for no in node:
                if isinstance(no, javalang.tree.Node):
                    index = node.index(no)
                    node[index] = type(no).__name__

            
    return level,nodes

In [269]:
# 使用javalang解析数据，将AST的层数,和ASTnodes(ast层次遍历结果)  变为dataframe中的一列，返回新的dataframe

def get_BFS_df(df):

    error_index = []
    asts = []
    levels = []
    nodes = []
    for i in range(len(df)):
        tokens_t = javalang.tokenizer.tokenize(df.loc[i]['new_code_raw'])
        try:
            parser = javalang.parse.Parser(tokens_t)
        except Exception as e:
            error_index.append(i)
            continue
        asts.append(parser)  

    for i in range(len(asts)):
        try:
            # 尝试解析Java代码
            test = asts[i].parse_member_declaration()
            depth,node = get_blocks_improved_BFS_code(test)
            levels.append(depth) 
            nodes.append(node)
        except javalang.parser.JavaSyntaxError:
            # 如果出现JavaSyntaxError，打印错误并跳过
            error_index.append(i)
            print(f"解析错误索引为ast[{i}]，跳过有问题的代码")
            continue 
            
    for i in error_index:
        df.drop(i, inplace=True)
    df = df.reset_index(drop=True)
    
    df['AST_level'] = levels
    df['AST_BFSnodes'] = nodes
    return df
test_df_ast = get_BFS_df(test_df)
test_df_ast.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,"[MethodDeclaration, None, {private, static}, [..."
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,"[MethodDeclaration, None, {public}, [], None, ..."
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,"[MethodDeclaration, None, {private}, [], None,..."
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,"[MethodDeclaration, None, {public, static}, []..."
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,"[MethodDeclaration, None, {static}, [], None, ..."


In [270]:
# 这个前序遍历 不能和层序遍历  在一个函数中连着使用 ， 最后用的是这个

def preorder_traversal_a(node,pre_nodes):
    if hasattr(node, 'children'):
        for child_node in node.children:
            pre_nodes.append(child_node)
            preorder_traversal_a(child_node,pre_nodes)
    else:
        try:
            len_node = len(node)
            for i in range(len_node):
                if hasattr(node[i], 'children'):
                    for child in node[i].children:
                        pre_nodes.append(child)
                        preorder_traversal_a(child,pre_nodes)
        except Exception as e:
            pass
    return pre_nodes

In [271]:
# 这个前序遍历 不能和层序遍历  在一个函数中连着使用 ， 最后用的是这个

def preorder_traversal_a_n(node,pre_nodes,depth=0):
    if depth == 0:
        pre_nodes.append(node)

    if hasattr(node, 'children'):
        for child_node in node.children:
            pre_nodes.append(child_node)
            preorder_traversal_a_n(child_node,pre_nodes,depth+1)
    else:
        try:
            len_node = len(node)
            for i in range(len_node):
                if hasattr(node[i], 'children'):
                    for child in node[i].children:
                        pre_nodes.append(child)
                        preorder_traversal_a_n(child,pre_nodes,depth+1)
        except Exception as e:
            pass
    return pre_nodes

In [272]:
# 获得前序遍历

def get_PFS_df(df):
    
    asts = []
    nodes_pfs = []
    for i in range(len(df)):
        tokens_t = javalang.tokenizer.tokenize(df.loc[i]['new_code_raw'])
        parser = javalang.parse.Parser(tokens_t)

        asts.append(parser)  

    for i in range(len(asts)):  
        try:
            # 尝试解析Java代码
#             print(i)
            test = asts[i].parse_member_declaration()

            pre_nodes_test = []
            node_p = preorder_traversal_a_n(test,pre_nodes_test)
            
            for node in node_p:   # 处理 node_p  
                if isinstance(node, javalang.tree.Node):
                    index = node_p.index(node)
                    node_p[index] = type(node).__name__
                if isinstance(node, list):
                    for no in node:
                        if isinstance(no, javalang.tree.Node):
                            index = node.index(no)
                            node[index] = type(no).__name__
                            
            nodes_pfs.append(node_p)
        except javalang.parser.JavaSyntaxError:
            continue 
    
    df['AST_PFSnodes'] = nodes_pfs

    return df
test_df_ast = get_PFS_df(test_df_ast)
test_df_ast.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes,AST_PFSnodes
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,"[MethodDeclaration, None, {private, static}, [...","[MethodDeclaration, None, {private, static}, [..."
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,"[MethodDeclaration, None, {public}, [], None, ...","[MethodDeclaration, None, {public}, [], None, ..."
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,"[MethodDeclaration, None, {private}, [], None,...","[MethodDeclaration, None, {private}, [], None,..."
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,"[MethodDeclaration, None, {public, static}, []...","[MethodDeclaration, None, {public, static}, []..."
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,"[MethodDeclaration, None, {static}, [], None, ...","[MethodDeclaration, None, {static}, [], None, ..."


In [273]:
# 可以把这个和下面的合并在一块
def get_clean_list(lst):
    lst = [x for x in lst if x is not None]
    lst = [x for x in lst if x != []]
    lst = [x for x in lst if x != '']
    return lst
# 去除多余的元素
def flatten_to_string(lst):
    result = []
    for i in lst:
        if isinstance(i, list):
            result.extend(flatten_to_string(i))
        elif isinstance(i, set):
            result.extend(flatten_to_string(list(i)))
        elif not isinstance(i, bool):
            result.append(i)
    return result

In [274]:
# 整理一下 将 里面的列表括号 和 多余元素删除  ( AST_PFSnodes 列和 AST_BFSnodes 列 )
def clean_ASTNodes(df):
    AST_BFSnodes = []
    AST_PFSnodes = []
    for node in df['AST_PFSnodes']:
        PFS_nodes = get_clean_list(node)
        PFS_nodes = flatten_to_string(PFS_nodes)
        AST_PFSnodes.append(PFS_nodes)
    for node in df['AST_BFSnodes']:
        BFS_nodes = get_clean_list(node)
        BFS_nodes = flatten_to_string(BFS_nodes)
        AST_BFSnodes.append(BFS_nodes)    
    
    df['AST_BFSnodes'] = AST_BFSnodes
    df['AST_PFSnodes'] = AST_PFSnodes
    return df
cleaned_test_df_ast = clean_ASTNodes(test_df_ast)
cleaned_test_df_ast.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes,AST_PFSnodes
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,"[MethodDeclaration, private, static, Reference...","[MethodDeclaration, private, static, Reference..."
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,"[MethodDeclaration, public, BasicType, areInse...","[MethodDeclaration, public, BasicType, boolean..."
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,"[MethodDeclaration, private, BasicType, marsha...","[MethodDeclaration, private, BasicType, byte, ..."
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,"[MethodDeclaration, public, static, ReferenceT...","[MethodDeclaration, public, static, ReferenceT..."
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,"[MethodDeclaration, static, ReferenceType, cre...","[MethodDeclaration, static, ReferenceType, Mon..."


In [275]:
cleaned_test_df_ast.AST_PFSnodes.apply(len).mean()

67.3

In [276]:
(cleaned_test_df_ast.AST_PFSnodes.apply(len) < 100).sum()

16

In [277]:
average_AST_level = cleaned_test_df_ast['AST_level'].mean()
average_AST_level

7.7

In [278]:
import re
import string

def split_punctuation(s):
    # 使用正则表达式匹配连续的标点符号或者字母和标点符号之间的位置
    splits = re.finditer(r'(?<=\w)(?=[{}])|(?<=[{}])(?=\w)'.format(string.punctuation, string.punctuation), s)
    
    # 获取所有分割位置
    split_positions = [match.start() for match in splits]
    
    # 在分割位置插入空格
    for pos in reversed(split_positions):
        s = s[:pos] + ' ' + s[pos:]
        
    s = s.replace("< s >", "<s>")
    s = s.replace("</ s >", "</s>")
    return s


In [279]:
# 获得父节点对应孩子的个数,前序遍历

def get_Child_Node_count(df):
    
    asts = []
    codes = []
    child_Node_count = []
    for i in range(len(df)):
        tokens_t = javalang.tokenizer.tokenize(df.loc[i]['new_code_raw'])
        parser = javalang.parse.Parser(tokens_t)
        
        codes.append(df.loc[i]['new_code_raw'])
        asts.append(parser)  

    for i in range(len(asts)):  
        try:
            # 尝试解析Java代码
#             print(i)
            test = asts[i].parse_member_declaration()

            pre_nodes_test = []
            node_p = preorder_traversal_a_n(test,pre_nodes_test)
            
            code = codes[i]
            tokens = split_punctuation(code)
            tokens = ' '.join(tokens.split()).split()
            
            pre_child_num = []
            for nodes in node_p:
                if hasattr(nodes, 'children'):
                    num = 0
                    for child_node in nodes.children:
                        if isinstance(child_node, javalang.tree.Node):
                            num = num + 1
                        elif isinstance(child_node, list):
                            num = num + len(child_node)
                        elif isinstance(child_node, set):
                            num = num + len(child_node)
                        elif child_node in tokens:
                            num = num + 1
                    pre_child_num.append(num)
                else:
                    try:
                        len_node = len(nodes)
                        for i in range(len_node):
                            if hasattr(nodes[i], 'children'):
                                num = 0
                                for child_node in nodes[i].children:
                                    if isinstance(child_node, javalang.tree.Node):
                                        num = num + 1
                                    elif isinstance(child_node, list):
                                        num = num + len(child_node)
                                    elif isinstance(child_node, set):
                                        num = num + len(child_node)
                                    elif child_node in tokens:
                                        num = num + 1
                                pre_child_num.append(num)
                    except Exception as e:
                        pass
            # 父节点的个数 + 子节点的个数 为树节点的个数
            # print(len(pre_child_num) )
            child_Node_count.append(pre_child_num)
        except javalang.parser.JavaSyntaxError:
            continue 
    
    df['Child_Node_count'] = child_Node_count

    return df
cleaned_test_df_ast_after_get_child = get_Child_Node_count(cleaned_test_df_ast)
cleaned_test_df_ast_after_get_child.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes,AST_PFSnodes,Child_Node_count
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,"[MethodDeclaration, private, static, Reference...","[MethodDeclaration, private, static, Reference...","[7, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 4, 1, 1, 1, ..."
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,"[MethodDeclaration, public, BasicType, areInse...","[MethodDeclaration, public, BasicType, boolean...","[4, 1, 1, 3, 3, 3, 2, 1, 3, 3, 2, 1]"
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,"[MethodDeclaration, private, BasicType, marsha...","[MethodDeclaration, private, BasicType, byte, ...","[10, 2, 2, 1, 2, 2, 2, 2, 2, 1, 3, 1, 0, 1, 1,..."
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,"[MethodDeclaration, public, static, ReferenceT...","[MethodDeclaration, public, static, ReferenceT...","[6, 2, 1, 1, 3, 1, 1, 4, 2, 0]"
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,"[MethodDeclaration, static, ReferenceType, cre...","[MethodDeclaration, static, ReferenceType, Mon...","[6, 1, 3, 3, 1, 1, 1, 2, 1, 1,

In [280]:
df_clean = cleaned_test_df_ast_after_get_child
df_clean.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes,AST_PFSnodes,Child_Node_count
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,"[MethodDeclaration, private, static, Reference...","[MethodDeclaration, private, static, Reference...","[7, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 4, 1, 1, 1, ..."
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,"[MethodDeclaration, public, BasicType, areInse...","[MethodDeclaration, public, BasicType, boolean...","[4, 1, 1, 3, 3, 3, 2, 1, 3, 3, 2, 1]"
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,"[MethodDeclaration, private, BasicType, marsha...","[MethodDeclaration, private, BasicType, byte, ...","[10, 2, 2, 1, 2, 2, 2, 2, 2, 1, 3, 1, 0, 1, 1,..."
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,"[MethodDeclaration, public, static, ReferenceT...","[MethodDeclaration, public, static, ReferenceT...","[6, 2, 1, 1, 3, 1, 1, 4, 2, 0]"
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,"[MethodDeclaration, static, ReferenceType, cre...","[MethodDeclaration, static, ReferenceType, Mon...","[6, 1, 3, 3, 1, 1, 1, 2, 1, 1,

In [281]:
len(df_clean)

20

In [282]:
# model = torch.load('save_GCBmodel.pt',map_location=torch.device('cuda:0'))
model = torch.load('D:/BERT_learing/CCDP/for_captum/save_model/save_GCBmodel.pt',map_location=torch.device('cpu'))

model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [283]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())  #输出为True，则安装无误


2.3.1+cpu
None
False


In [284]:
device

device(type='cpu')

In [285]:
tokenizer = RobertaTokenizer.from_pretrained("D:/BERT_learing/code_comment_inconsistency_detection/graphcodebert")

In [286]:
model.roberta.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(50265, 768, padding_idx=1)
  (position_embeddings): Embedding(514, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [287]:
model.roberta.encoder.layer[0]

RobertaLayer(
  (attention): RobertaAttention(
    (self): RobertaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): RobertaSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): RobertaIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): RobertaOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [288]:
input_embeddings = model.get_input_embeddings()

In [289]:
# predict和squad_pos_forward_func可以合成一个
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                  attention_mask=attention_mask )
    
    prediction = output.logits
    prediction_1 = nn.functional.softmax(prediction, dim=1)
    prediction = prediction_1.max(1).values
    out = torch.argmax(prediction_1, dim=-1)
    # prediction：每个输入样本的最大预测概率。
    # out：预测的类别标签。
    # prediction_1：所有类别的预测概率。    
    return prediction,out,prediction_1

In [290]:
def squad_pos_forward_func(inputs,position_ids=None, attention_mask=None, position=0):
    pred ,_,_= predict(inputs,
                     position_ids=position_ids,
                   attention_mask=attention_mask)
    return pred

In [291]:
ref_token_id = tokenizer.pad_token_id
sep_token_id = tokenizer.sep_token_id
cls_token_id = tokenizer.cls_token_id
ref_token_id,sep_token_id,cls_token_id

(1, 2, 0)

In [292]:
# 注意长度
def truncate(ids,len_tru = 512):
    return ids[:len_tru] if len(ids) > len_tru else ids

In [293]:
# 这是单个数据的处理方式，应该要想数据集应该怎么处理
def construct_input_ref_pair(comment,AST_type,  ref_token_id, sep_token_id, cls_token_id):
    comment = tokenizer.encode(comment, add_special_tokens=False,truncation=True,max_length=512)
    AST_type = tokenizer.encode(AST_type, add_special_tokens=False,truncation=True,max_length=512)
    # construct input token ids
    input_ids = [cls_token_id] + comment + [sep_token_id] + AST_type + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(comment) + [sep_token_id] + \
        [ref_token_id] * len(AST_type) + [sep_token_id]
    input_ids = truncate(input_ids)
    ref_input_ids = truncate(ref_input_ids)
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(comment)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = model.roberta.embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = model.roberta.embeddings(ref_input_ids, token_type_ids=ref_token_type_ids, position_ids=ref_position_ids)
    
    return input_embeddings, ref_input_embeddings

In [294]:
def input_data_list(AST_list,comment_list):
    input_ids_all = []
    ref_input_ids_all = []
    position_ids_all = []
    attention_mask_all = []
    token_type_ids_all = []
    all_tokens_all = []
    for i in range(len(AST_list)):
        input_ids, ref_input_ids, comment_len = construct_input_ref_pair(comment_list[i],AST_list[i], ref_token_id, sep_token_id, cls_token_id)
        token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, comment_len)
        position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
        attention_mask = construct_attention_mask(input_ids)
        
        indices = input_ids[0].detach().tolist()
        
        all_tokens = []                                       ###
        for _, token in enumerate(indices):
            all_tokens.append(tokenizer.decode([token]))
        
        input_ids_all.append(input_ids)
        ref_input_ids_all.append(ref_input_ids)
        position_ids_all.append(position_ids)
        attention_mask_all.append(attention_mask)
        token_type_ids_all.append(token_type_ids)
        all_tokens_all.append(all_tokens)

    return input_ids_all,ref_input_ids_all,position_ids_all,attention_mask_all,token_type_ids_all,all_tokens_all 


In [295]:
import javalang.tree

# 获取javalang.tree模块中的所有类
# 树节点的所有名称
node_classes = [cls for cls in dir(javalang.tree) if isinstance(getattr(javalang.tree, cls), type)]

print(node_classes,len(node_classes))

['Annotation', 'AnnotationDeclaration', 'AnnotationMethod', 'ArrayCreator', 'ArrayInitializer', 'ArraySelector', 'AssertStatement', 'Assignment', 'BasicType', 'BinaryOperation', 'BlockStatement', 'BreakStatement', 'Cast', 'CatchClause', 'CatchClauseParameter', 'ClassCreator', 'ClassDeclaration', 'ClassReference', 'CompilationUnit', 'ConstantDeclaration', 'ConstructorDeclaration', 'ContinueStatement', 'Creator', 'Declaration', 'DoStatement', 'Documented', 'ElementArrayValue', 'ElementValuePair', 'EnhancedForControl', 'EnumBody', 'EnumConstantDeclaration', 'EnumDeclaration', 'ExplicitConstructorInvocation', 'Expression', 'FieldDeclaration', 'ForControl', 'ForStatement', 'FormalParameter', 'IfStatement', 'Import', 'InferredFormalParameter', 'InnerClassCreator', 'InterfaceDeclaration', 'Invocation', 'LambdaExpression', 'Literal', 'LocalVariableDeclaration', 'Member', 'MemberReference', 'MethodDeclaration', 'MethodInvocation', 'MethodReference', 'Node', 'PackageDeclaration', 'Primary', 'Ref

In [296]:
# 构建树，用于计算父节点归因值
class Node:
    def __init__(self, data):
        self.data = data
        self.children = []

def build_tree(attribution_num, child_num):
    # 初始化一个栈
    stack = []
    #  attribution_num [index]
    index = 0
    # 创建根节点并入栈
    root = Node(attribution_num[0])
    # 收集所有的父节点
    parents = [root]  
    
    stack.append((root, child_num[index]))
    index += 1
    # 遍历 attribution_num
    for i in range(1, len(attribution_num)):
        
        # 创建一个新的节点
        node = Node(attribution_num[i])
        
        # 如果栈顶的节点还有子节点的位置，那么将新的节点添加为栈顶节点的子节点
        if stack[-1][1] > 0 and attribution_num[i] != 0 :
            stack[-1][0].children.append(node)
            stack[-1] = (stack[-1][0], stack[-1][1] - 1)
            
        # 如果新的节点是一个父节点，那么将其压入栈中
        if attribution_num[i] == 0 and stack[-1][1] > 0 and index < len(child_num):
            
            stack[-1][0].children.append(node)
            stack[-1] = (stack[-1][0], stack[-1][1] - 1)
            
            
            parents.append(node)
            stack.append((node, child_num[index]))
            index += 1
            
        # 如果栈顶的节点没有子节点的位置了，那么将其从栈中移除
        while stack and stack[-1][1] == 0:
            stack.pop()
    return root,parents

# 计算父节点归因值 ，这个对，用这个
def calculate_parent_value(node):
    if node.children:
        node.data = sum(calculate_parent_value(child) for child in node.children) / len(node.children)
    return node.data

In [297]:
# root,parents = build_tree(num_attr, num_child)

# parent_values = []

# calculate_parent_value(root)

# # 返回所有父节点的值
# parent_values = [node.data for node in parents]
# print(parent_values)

In [298]:
# 前k个贡献最高的word 和 token_type 和 position
# return value为归因贡献值  indices为词对应的索引  top_tokens为 词或位置或token_type
def get_topk_attributed_tokens(attrs,all_token_t, k=5):
    values_max, indices_max = torch.topk(attrs, k)
    top_tokens_max = [all_token_t[idx] for idx in indices_max]
    values_min, indices_min = torch.topk(attrs, k, largest=False)
    top_tokens_min = [all_token_t[idx] for idx in indices_min] 
    
    return top_tokens_max, values_max, indices_max,top_tokens_min,values_min,indices_min


In [299]:
# list_t_1 = token
# list_t_2 = attribute

# list_t_1 为 已经转化token的单词， list_t_2 为 attribute
def get_ast_num(nodes,list_t_1,list_t_2):
    attr = []    
    for node in nodes:
        try:
            index = list_t_1.index(node)       
            del list_t_1[index]
            
            attr.append(list_t_2[index])
            del list_t_2[index]

        except Exception as e:
            attr.append(0)
    return nodes,attr

In [300]:
# 从 all_tokens 还原为 原单词 ，并且计算归因值
def get_restore_words(code,comment,input_ids,all_tokens,attribution_num):
    all_tokens_decode = tokenizer.decode(input_ids)
    len_all_tokens_decode = len(all_tokens_decode) + 4
    # 使用decode获得的序列，去掉分词之后的空格    例 ' a' -> 'a'
    all_tokens_clean = []
    for token in all_tokens:
        s_without_leading_space = token.lstrip()
        all_tokens_clean.append(s_without_leading_space)
#     print('all_tokens_clean:\n',all_tokens_clean)
#     print('all_tokens_clean:\n',len(all_tokens_clean))
    

    # 获得 code_comment_baseline
    code = tokenizer.encode(code, add_special_tokens=False,truncation=True,max_length=512)
    comment = tokenizer.encode(comment, add_special_tokens=False,truncation=True,max_length=512)
    code_decode = tokenizer.decode(code)
    comment_decode = tokenizer.decode(comment)
    code_comment_baseline = tokenizer.decode(tokenizer.cls_token_id) + ' '+ comment_decode + ' '+ tokenizer.decode(tokenizer.sep_token_id) + ' ' + code_decode + ' ' + tokenizer.decode(tokenizer.sep_token_id)
    code_comment_baseline = code_comment_baseline[:len_all_tokens_decode]
    code_comment_baseline = split_punctuation(code_comment_baseline)
    code_comment_baseline = code_comment_baseline.split()

#     print('code_comment_baseline:\n',code_comment_baseline)
#     print('code_comment_baseline_len:\n',len(code_comment_baseline))

    
    # 获得 相邻有几个token合并在一块的列表times  为了以后再计算attribute时求和
    times = []
    token_index = 0
    for code_comment in code_comment_baseline:
        temp = ''
        time = 0
        while temp != code_comment:
            temp = temp + all_tokens_clean[token_index]
            token_index = token_index + 1
            time = time + 1
        times.append(time)
#     print('times:\n',times)
    
    attribute_sum = []
    start = 0
    for time in times:
        end = start + time
        attribute = sum(attribution_num[start:end]) / time
        attribute_sum.append(attribute)
        start = end
#     print('attribute_sum:\n',attribute_sum)
    return code_comment_baseline ,attribute_sum 

In [301]:
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

lig = LayerIntegratedGradients(squad_pos_forward_func,input_embeddings)

将父节点和归因值进行统计分析

In [302]:
def interpret_sentence_3(code,comment,PFS,child_num,input_ids,ref_input_ids, token_type_ids,\
                         position_ids, attention_mask, all_tokens):
    pre ,out,_ = predict(input_ids, position_ids=position_ids, attention_mask=attention_mask)

    attributions_ig, delta_ig = lig.attribute(input_ids, baselines=ref_input_ids,\
                           additional_forward_args=(position_ids,attention_mask,0),return_convergence_delta=True,internal_batch_size=8)
    
    attributions = attributions_ig.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
#     print(delta_ig)
#     print(attributions)
    try:
        all_tokens ,attributions = get_restore_words(code,comment,input_ids[0],all_tokens,attributions)     # 合并为一个单词
        attributions = torch.tensor(attributions)
       
        attributions_copy = attributions.tolist()
        all_tokens_copy = all_tokens

        nodes,attr = get_ast_num(PFS,all_tokens_copy,attributions_copy)
        # 保留5为小数
        attr = [round(num, 5) for num in attr]
        
        root,parents = build_tree(attr, child_num)
        parent_values = []
        calculate_parent_value(root)
        # 返回所有父节点的值
        parent_values = [node.data for node in parents]

        # 找到parent_value对应的字符，例如 MethodDeclaration：320
        parent_node_name = [nodes[i] for i in range(len(nodes)) if attr[i] == 0]
        
        return out,parent_node_name,parent_values,nodes,attr
    except Exception as e:
        print("解析错误")
        return -1,_,_,_,_

In [303]:
def analyse_ASTNode_attribution(df):
    code_list = df['new_code_raw']
    comment_list = df['old_comment_raw']
    ground_lable = df['label']
    PFS_list = df['AST_PFSnodes']
    child_num_list = df['Child_Node_count']
    
    input_ids_all,ref_input_ids_all,position_ids_all,attention_mask_all,\
    token_type_ids_all,all_tokens_all= input_data_list(code_list,comment_list)
    
    parent_node_name_true = []
    parent_values_true = []
    index_true = []
    nodes_true = []
    attr_true = []
    
    parent_node_name_false = []
    parent_values_false = []
    index_false = []
    nodes_false = []
    attr_false = []

    
    for i in range(len(code_list)): 
#         print(i)
        out,parent_node_name,parent_values,nodes,attr\
        = interpret_sentence_3(code_list[i],comment_list[i],PFS_list[i],child_num_list[i],\
                               input_ids_all[i],ref_input_ids_all[i], token_type_ids_all[i],\
                               position_ids_all[i], attention_mask_all[i], all_tokens_all[i])
#        解析错误的情况不能直接判断是否相等
        if out != -1:
            if out == ground_lable[i]:
                index_true.append(i)
                parent_node_name_true.append(parent_node_name)
                parent_values_true.append(parent_values)
                nodes_true.append(nodes)
                attr_true.append(attr)
            else :
                index_false.append(i)
                parent_node_name_false.append(parent_node_name)
                parent_values_false.append(parent_values)
                nodes_false.append(nodes)
                attr_false.append(attr)

    return index_true,index_false,parent_node_name_true,parent_values_true,nodes_true,attr_true,\
                                  parent_node_name_false,parent_values_false,nodes_false,attr_false

In [304]:
index_true,index_false,\
parent_node_name_true,parent_values_true,nodes_true,attr_true,\
parent_node_name_false,parent_values_false,nodes_false,attr_false\
= analyse_ASTNode_attribution(df_clean)

解析错误
解析错误
解析错误
解析错误


In [305]:
# 解决 parent_node_name 与 parent_values 数量不一致问题
def clean_parent_node(parent_node_name,node_class):
    final_parent_node = []
    for parent_node in parent_node_name:
        new_parent_node = []
        for p in parent_node:
            if p in node_class:
                new_parent_node.append(p)
        final_parent_node.append(new_parent_node)  
    return final_parent_node
# node_classes 为 javalang库中树节点的所有成员名称
parent_node_name_true = clean_parent_node(parent_node_name_true,node_classes)
parent_node_name_false = clean_parent_node(parent_node_name_false,node_classes)

In [306]:
len(parent_node_name_true),len(index_true)

(9, 9)

In [307]:
index_true

[1, 3, 4, 7, 13, 16, 17, 18, 19]

In [308]:
# 可以再分析一下原因写进文章里,后六个是我自己加的     这个还得在过滤一下
# 用 8 - 10 个就行 画图之后挑出好看的，或者对比幅度大一点的
after_filter_name = ['MethodDeclaration','VariableDeclarator','StatementExpression','IfStatement',
                    'ForStatement','WhileStatement','TryStatement', 'ReturnStatement']

Not_delet_AST_token_type = ['FormalParameter','ClassReference', 'BasicType', 'InterfaceDeclaration',
                            'CatchClauseParameter', 'MethodInvocation', 'SuperMethodInvocation', 
                            'SuperMemberReference', 'ConstructorDeclaration', 'ReferenceType',
                            'MethodDeclaration', 'VariableDeclarator', 'IfStatement',
                            'WhileStatement', 'DoStatement', 'ForStatement', 'AssertStatement', 
                            'BreakStatement', 'ContinueStatement', 'ReturnStatement', 'ThrowStatement',
                            'SynchronizedStatement', 'TryStatement', 'SwitchStatement', 'BlockStatement', 
                            'StatementExpression', 'TryResource', 'CatchClause', 'CatchClauseParameter',
                            'SwitchStatementCase', 'ForControl', 'EnhancedForControl','BinaryOperation',
                            'UnaryOperation','TernaryExpression','SynchronizedStatement']

In [309]:
def get_statement(parent_type,attribution_num,filter_type,index_judge,average_list,nodes,attr):    
    list_pair = [list(zip(sublist1, sublist2)) for sublist1, sublist2 in zip(parent_type, attribution_num)]
    
    type_dict = dict(average_list)

    filtered_nested_statements = []
    removed_indices = []
    
    for index, sublist in enumerate(list_pair):
        if all(stmt_type in type_dict and value > type_dict[stmt_type] for stmt_type, value in sublist if stmt_type in type_dict):
            filtered_nested_statements.append(sublist)
        else:
            removed_indices.append(index)
#     print(removed_indices)
    index_all = [index_judge[i] for i in range(len(index_judge)) if i not in removed_indices]
    nodes_list = [element for i, element in enumerate(nodes) if i not in removed_indices]
    attrs_list = [element for i, element in enumerate(attr) if i not in removed_indices]

    
    return filtered_nested_statements,index_all,nodes_list,attrs_list

true

In [312]:
# 由均值而来 iqr
type_list_true = [
    ('MethodDeclaration',0.14),
    ('VariableDeclarator',0.055),
    ('ForStatement',0.058),
    ('ReturnStatement',0.073)
]

In [ ]:
# 由均值而来 Winsorization 
type_list_true_1 = [
    ('MethodDeclaration',0.16),
    ('VariableDeclarator',0.061),
    ('ForStatement',0.057),
    ('ReturnStatement',0.091)
]

In [313]:
# after_filter 
filtered_statements_true,filter_true_index,filter_nodes_true,filter_attr_true\
= get_statement(parent_node_name_true,parent_values_true,after_filter_name,index_true,type_list_true,nodes_true,attr_true)

In [314]:
filtered_statements_true

[[('MethodDeclaration', 0.049),
  ('BasicType', -0.01659),
  ('ReturnStatement', 0.05866),
  ('BinaryOperation', 0.05866),
  ('BinaryOperation', 0.09469),
  ('BinaryOperation', 0.05418),
  ('MethodInvocation', 0.011170000000000001),
  ('Literal', -0.02782),
  ('MethodInvocation', 0.07203),
  ('BinaryOperation', 0.04099),
  ('MethodInvocation', -0.0046),
  ('Literal', 0.09981)],
 [('MethodDeclaration', 0.013996333333333333),
  ('ReferenceType', 0.05356),
  ('IfStatement', 0.04955166666666666),
  ('ReturnStatement', 0.047356666666666665),
  ('BinaryOperation', 0.047356666666666665),
  ('MemberReference', 0.07384),
  ('Literal', 0.02543),
  ('BlockStatement', 0.05174666666666666),
  ('StatementExpression', 0.05174666666666666),
  ('Assignment', 0.05174666666666666),
  ('MemberReference', -0.01323),
  ('ClassCreator', 0.01353),
  ('ReferenceType', 0.01353),
  ('MemberReference', 0.00358)],
 [('MethodDeclaration', 0.048474166666666665),
  ('BasicType', 0.05739),
  ('ReturnStatement', 0.0575

In [315]:
filter_true_index

[1, 7, 13]

In [335]:
len(filtered_statements_true),len(filter_true_index) 

(3, 3)

In [336]:
def get_json(df,filtered_statements,filter_index,filtered_nodes,filtered_attrs,name):
    filtered_df = df.loc[filter_index]
    filtered_df['all_nodes'] = filtered_nodes
    filtered_df['all_attr'] = filtered_attrs
    filtered_df["raw_data_index_column"] = filter_index
    filtered_df['filtered_statements_attr'] = filtered_statements
    
    filtered_df.to_json('asttype_attribution_index_'+name+'.json', orient='records')
    return filtered_df

In [337]:
filtered_df_true = get_json(df_clean,filtered_statements_true,filter_true_index,filter_nodes_true,filter_attr_true,name='true')

false

In [338]:
# 由均值而来 iqr
type_list_false = [
    ('MethodDeclaration',0.167),
    ('WhileStatement',0.067),
    ('TryStatement',0.063),
    ('ReturnStatement',0.101)
]

In [ ]:
# 由均值而来 Winsorization 
type_list_false_1 = [
    ('MethodDeclaration',0.191),
    ('ForStatement',0.07),
    ('WhileStatement',0.066),
    ('ReturnStatement',0.121)
]

In [339]:
# after_filter 
filtered_statements_false,filter_false_index,filter_nodes_false,filter_attr_false\
= get_statement(parent_node_name_false,parent_values_false,after_filter_name,index_false,type_list_false,nodes_false,attr_false)

In [340]:
filter_false_index

[0, 5, 10, 14]

In [341]:
filtered_df_false = get_json(df_clean,filtered_statements_false,filter_false_index,filter_nodes_false,filter_attr_false,name='false')
filtered_df_false.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,...,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes,AST_PFSnodes,Child_Node_count,all_nodes,all_attr,raw_data_index_column,filtered_statements_attr
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",...,"[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,"[MethodDeclaration, private, static, Reference...","[MethodDeclaration, private, static, Reference...","[7, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 4, 1, 1, 1, ...","[MethodDeclaration, private, static, Reference...","[0, 0.0378, 0.01513, 0, -0.01173, -0.03101, 0,...",0,"[(MethodDeclaration, 0.022379315476190476), (R..."
5,debezium_debezium-1182-FirstSentence-0,0,Summary,Construct the value of the data attribute of C...,"[construct, the, value, of, the, data, attribu...",Construct the value of the data attribute of C...,"[construct, the, value, of, the, data, attribu...",[],public Struct ceDataAttribute() {\n ...,"[public, struct, ce, data, attribute, (, ), {,...",...,"[<KEEP>, public, struct, ce, data, attribute, ...","[<KEEP>, public, <KEEP>, struct, <KEEP>, ce, <...",4,"[MethodDeclaration, public, ReferenceType, ceD...","[MethodDeclaration, public, ReferenceType, Str...","[4, 1, 1, 2]","[MethodDeclaration, public, ReferenceType, Str...","[0, 0.07269, 0, 0.48757, 0.25655, 0, 0, 0.1204...",5,"[(MethodDeclaration, 0.24018125), (ReferenceTy..."
10,apache_jclouds-356-Associations-FirstSentence,1,Summary,Returns the providers that are bound to the sa...,"[returns, the, providers, that, are, bound, to...",Returns the providers that are bound to the sa...,"[returns, the, providers, that, are, bound, to...","[<REPLACE_OLD>, type, <REPLACE_NEW>, context, ...","public static Iterable<ProviderMetadata<?, ...","[public, static, iterable, <, provider, metada...",...,"[<KEEP>, public, static, iterable, <, provider...","[<KEEP>, public, <KEEP>, static, <KEEP>, itera...",7,"[MethodDeclaration, public, static, ReferenceT...","[MethodDeclaration, public, static, ReferenceT...","[7, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 3, 1, 4, 3, ...","[MethodDeclaration, public, static, ReferenceT...","[0, -0.01273, -0.0393, 0, -0.01854, 0, 0, -0.0...",10,"[(MethodDeclaration, 0.00842151455026455), (Re..."
14,reddit_Lob-0-Associations-FirstSentence,1,Summary,Build a Ravioli client,"[build, a, ravioli, client]",Build a Lob client,"[build, a, lob, client]","[<REPLACE_OLD>, ravioli, <REPLACE_NEW>, lob, <...",public Ravioli build() {\n ...,"[public, ravioli, build, (, ), {, return, new,...",...,"[<KEEP>, public, <KEEP_END>, <REPLACE_OLD>, ra...","[<KEEP>, public, <REPLACE_OLD>, ravioli, <REPL...",5,"[MethodDeclaration, public, ReferenceType, bui...","[MethodDeclaration, public, ReferenceType, Lob...","[4, 1, 1, 2, 1, 0]","[MethodDeclaration, public, ReferenceType, Lob...","[0, 0.01499, 0, 0.23625, 0.20671, 0, 0, 0, 0.3...",14,"[(MethodDeclaration, 0.15595375), (ReferenceTy..."


In [342]:
df_true = pd.read_json('asttype_attribution_index_true.json')
df_false = pd.read_json('asttype_attribution_index_false.json')
df_true.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,...,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes,AST_PFSnodes,Child_Node_count,all_nodes,all_attr,raw_data_index_column,filtered_statements_attr
0,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",...,"[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,"[MethodDeclaration, public, BasicType, areInse...","[MethodDeclaration, public, BasicType, boolean...","[4, 1, 1, 3, 3, 3, 2, 1, 3, 3, 2, 1]","[MethodDeclaration, public, BasicType, boolean...","[0, 0.16561, 0, -0.01659, -0.01168000000000000...",1,"[[MethodDeclaration, 0.049], [BasicType, -0.01..."
1,zaproxy_zaproxy-2180-FirstSentence-0,0,Summary,This method initializes txtPattern,"[this, method, initializes, txt, pattern]",This method initializes txtPattern,"[this, method, initializes, txt, pattern]",[],\tpublic JTextField getTxtPattern() {\n\t\tif ...,"[public, jtext, field, get, txt, pattern, (, )...",...,"[<KEEP>, public, <KEEP_END>, <REPLACE_OLD>, jt...","[<KEEP>, public, <REPLACE_OLD>, jtext, <REPLAC...",8,"[MethodDeclaration, public, ReferenceType, get...","[MethodDeclaration, public, ReferenceType, Zap...","[5, 1, 2, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1]","[MethodDeclaration, public, ReferenceType, Zap...","[0, -0.02485, 0, 0.05356, -0.01186, 0, 0, 0, 0...",7,"[[MethodDeclaration, 0.0139963333], [Reference..."
2,apache_kylin-447-FirstSentence-0,0,Summary,"HBase region cut size, in GB","[hbase, region, cut, size, ,, in, gb]","HBase region cut size, in GB","[hbase, region, cut, size, ,, in, gb]",[],public float getKylinHBaseRegionCut() {\n ...,"[public, float, get, kylin, hbase, region, cut...",...,"[<KEEP>, public, float, get, kylin, hbase, reg...","[<KEEP>, public, <KEEP>, float, <KEEP>, get, <...",6,"[MethodDeclaration, public, BasicType, getKyli...","[MethodDeclaration, public, BasicType, float, ...","[4, 1, 1, 3, 3, 0, 0]","[MethodDeclaration, public, BasicType, float, ...","[0, 0.10171000000000001, 0, 0.0573900000000000...",13,"[[MethodDeclaration, 0.0484741667], [BasicType..."


In [343]:
df_false.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,...,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes,AST_PFSnodes,Child_Node_count,all_nodes,all_attr,raw_data_index_column,filtered_statements_attr
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",...,"[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,"[MethodDeclaration, private, static, Reference...","[MethodDeclaration, private, static, Reference...","[7, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 4, 1, 1, 1, ...","[MethodDeclaration, private, static, Reference...","[0, 0.0378, 0.015130000000000001, 0, -0.01173,...",0,"[[MethodDeclaration, 0.0223793155], [Reference..."
1,debezium_debezium-1182-FirstSentence-0,0,Summary,Construct the value of the data attribute of C...,"[construct, the, value, of, the, data, attribu...",Construct the value of the data attribute of C...,"[construct, the, value, of, the, data, attribu...",[],public Struct ceDataAttribute() {\n ...,"[public, struct, ce, data, attribute, (, ), {,...",...,"[<KEEP>, public, struct, ce, data, attribute, ...","[<KEEP>, public, <KEEP>, struct, <KEEP>, ce, <...",4,"[MethodDeclaration, public, ReferenceType, ceD...","[MethodDeclaration, public, ReferenceType, Str...","[4, 1, 1, 2]","[MethodDeclaration, public, ReferenceType, Str...","[0, 0.07269, 0, 0.48757000000000006, 0.25655, ...",5,"[[MethodDeclaration, 0.24018125], [ReferenceTy..."
2,apache_jclouds-356-Associations-FirstSentence,1,Summary,Returns the providers that are bound to the sa...,"[returns, the, providers, that, are, bound, to...",Returns the providers that are bound to the sa...,"[returns, the, providers, that, are, bound, to...","[<REPLACE_OLD>, type, <REPLACE_NEW>, context, ...","public static Iterable<ProviderMetadata<?, ...","[public, static, iterable, <, provider, metada...",...,"[<KEEP>, public, static, iterable, <, provider...","[<KEEP>, public, <KEEP>, static, <KEEP>, itera...",7,"[MethodDeclaration, public, static, ReferenceT...","[MethodDeclaration, public, static, ReferenceT...","[7, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 3, 1, 4, 3, ...","[MethodDeclaration, public, static, ReferenceT...","[0, -0.012730000000000002, -0.0393, 0, -0.0185...",10,"[[MethodDeclaration, 0.0084215146], [Reference..."
3,reddit_Lob-0-Associations-FirstSentence,1,Summary,Build a Ravioli client,"[build, a, ravioli, client]",Build a Lob client,"[build, a, lob, client]","[<REPLACE_OLD>, ravioli, <REPLACE_NEW>, lob, <...",public Ravioli build() {\n ...,"[public, ravioli, build, (, ), {, return, new,...",...,"[<KEEP>, public, <KEEP_END>, <REPLACE_OLD>, ra...","[<KEEP>, public, <REPLACE_OLD>, ravioli, <REPL...",5,"[MethodDeclaration, public, ReferenceType, bui...","[MethodDeclaration, public, ReferenceType, Lob...","[4, 1, 1, 2, 1, 0]","[MethodDeclaration, public, ReferenceType, Lob...","[0, 0.014990000000000002, 0, 0.236250000000000...",14,"[[MethodDeclaration, 0.15595375], [ReferenceTy..."


用上述json与linelevel找交集，挑数据